# Tracing markers backwards in time

<img src='../../figures/reversetime.png' alt='thumbnail' width='200'/>

This example shows how to trace markers backwards in time.

For starters it is worth emphasizing that **this tutorial is only for tracing marker orbits backwards in time and not for reverting the physical process**.
Activating the time reversal option, ``REVERSE_TIME=1``, makes the time run backwards in simulations so that one can e.g. see the orbit at which a marker hit the wall or FILD.
However, it does not reverse collisional or other stochastic process so these must be disabled.
Use BMC for proper backwards Monte Carlo modelling that include these processess and can properly run a slowing-down simulation in reverse.

First set up the test data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)
a5.data.create_input("bfield analytical iter circular")
a5.data.create_input("plasma flat")
a5.data.create_input("E_TC")
a5.data.create_input("N0_1D")
a5.data.create_input("Boozer")
a5.data.create_input("MHD_STAT")
a5.data.create_input("asigma_loc")

print("Inputs created")

As for the wall, any wall data would do but we wish to do some nice visualizations in 3D so we choose a 3D wall for this exercise.

In [ ]:
from a5py.ascot5io.wall import wall_3D

rad  = 2.0
pol  = np.linspace(0, 2*np.pi, 181)[:-1]
w2d = {"nelements":180,
       "r":6.4 + rad*np.cos(pol), "z":rad*np.sin(pol)}
w3d = wall_3D.convert_wall_2D(180, **w2d)
a5.data.create_input("wall_3D", **w3d, desc="REVOLVED")

Now let's create a few markers which we trace for a few moments as if we were modelling the slowing-down process.
This is an ordinary forward in time simulation, we will use the results to run the reversed simulation (although there is nothing preventing from running the reversed simulation without the forward simulation).

In [ ]:
from a5py.ascot5io.marker import Marker
nmrk = 10
mrk = Marker.generate("gc", n=nmrk, species="alpha")
mrk["energy"][:] = 3.5e6
mrk["pitch"][:]  = 0.6
mrk["r"][:]      = np.linspace(7.6,8.0,nmrk)
a5.data.create_input("gc", **mrk, activate=True)

from a5py.ascot5io.options import Opt

opt = Opt.get_default()
opt.update({
    # Reversed time does not work well with the adaptive step at the moment so disable it
    "SIM_MODE":2, "ENABLE_ADAPTIVE":0,
    "ENDCOND_SIMTIMELIM":1, "ENDCOND_LIM_SIMTIME":1.0e-4, "ENDCOND_WALLHIT":1,
    "ENABLE_ORBIT_FOLLOWING":1, "ENABLE_COULOMB_COLLISIONS":1,
    "ENABLE_ORBITWRITE":1, "ORBITWRITE_MODE":1, "ORBITWRITE_INTERVAL":0,
    "ORBITWRITE_NPOINT":10**4,
})
a5.data.create_input("opt", **opt, desc="TIMEFORWARD", activate=True)

Run the simulation

In [ ]:
import subprocess
subprocess.run(["./../../build/ascot5_main", "--d=\"GREATESTHITSVOL2\""])
print("Simulation completed")

Few markers hit the wall.
We select one of them and run the orbit backwards in time.

In [ ]:
a5 = Ascot("ascot.h5")

# Pick one of the markers that hit the wall
ids = a5.data.active.getstate("ids", endcond="wall")[0]

# When converting endstate to marker input, the input type and the simulation
# mode that was used should preferably match
mrk = a5.data.active.getstate_markers("gc", ids=ids)

opt = a5.data.active.options.read()
opt.update({
    # Reverse time and turn off the collisions
    "REVERSE_TIME":1, "ENABLE_COULOMB_COLLISIONS":1,
    # When using the reversed time, the simulation is stopped if time < LIM_SIMTIME.
    # To follow marker back to its original position, we set LIM_SIMTIME=0 which was
    # the marker time at the start of the forward simulation.
    # Note that MAX_MILEAGE works same way both in forward and reversed simulations.
    "ENDCOND_LIM_SIMTIME":0.0,
    "ENDCOND_WALLHIT":0
})

Since we are only tracing a single marker, and we are only interested in its orbit, the reversed simulation can be done conveniently using the virtual run (but ordinary run would work as well).

Run the simulation and plot both the forward orbit and the reversed orbit.

In [ ]:
a5.simulation_initinputs()
a5.simulation_initmarkers(**mrk)
a5.simulation_initoptions(**opt)
vrun = a5.simulation_run(printsummary=False)

fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

a5.data.active.plotorbit_trajectory("r", "z", ids=mrk["ids"], axes=ax1)
a5.input_plotwallcontour(axes=ax1)

vrun.plotorbit_trajectory("r", "z", axes=ax2)
a5.input_plotwallcontour(axes=ax2)

plt.show()

This is how time reversed simulations are done.
They can be used to map the phase-space region where losses originate (for collisionless losses) or to make nice visualizations of marker orbits when they hit the wall or FILD.

In [ ]:
fig, ax = plt.subplots()
vrun.plotwall_3dstill(cpos=(-7.0,0.0,0.5), cfoc=(-3.8,-3.5,1.5), cang=(0,-20,-90), orbit=mrk["ids"], axes=ax)
a5.simulation_free()